In [1]:
### import libraries

import pandas as pd
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

In [25]:
def cambiar_valores_preservando_proporcion(df, columna_a_cambiar, fraccion_a_cambiar):
    # Verifica que la columna especificada exista en el DataFrame
    if columna_a_cambiar not in df.columns:
        print(f"La columna '{columna_a_cambiar}' no existe en el DataFrame.")
        return None

    # Verifica que la fracción esté en el rango [0, 1]
    if fraccion_a_cambiar < 0 or fraccion_a_cambiar > 1:
        print("La fracción a cambiar debe estar en el rango [0, 1].")
        return None

    # Calcula la cantidad de filas a cambiar basándose en la fracción proporcionada
    cantidad_filas_cambiar = int(fraccion_a_cambiar * len(df))
    print(cantidad_filas_cambiar)

    # Filtra el DataFrame por la clase especificada
    df_clase_permit = df[df[columna_a_cambiar] == 1]    
    df_clase_deny = df[df[columna_a_cambiar] == 0]

    # Verifica que haya suficientes filas para cambiar en ambas clases
    if cantidad_filas_cambiar > min(len(df_clase_permit), len(df_clase_deny)):
        print("No hay suficientes filas en una o ambas clases para cambiar.")
        return None

    # Selecciona aleatoriamente las filas a cambiar en cada clase
    filas_cambiar_permit = np.random.choice(df_clase_permit.index, size=int(cantidad_filas_cambiar / 2), replace=False)
    filas_cambiar_deny = np.random.choice(df_clase_deny.index, size=int(cantidad_filas_cambiar / 2), replace=False)

    # Cambia los valores de las filas seleccionadas
    df.loc[filas_cambiar_permit, columna_a_cambiar] = 0
    print("+ a cambiar=", len(filas_cambiar_permit))
    df.loc[filas_cambiar_deny, columna_a_cambiar] = 1
    print("- a cambiar=", len(filas_cambiar_deny))

    # Agrega la columna "flag" para la trazabilidad
    df['flag'] = 0
    df.loc[np.concatenate([filas_cambiar_permit, filas_cambiar_deny]), 'flag'] = 1

    return df, cantidad_filas_cambiar

def quedarse_con_mayor_freq(dataframe):
    # Ordenar el DataFrame por 'freq' en orden descendente
    dataframe_sorted = dataframe.sort_values(by='freq', ascending=False)

    # Quedarse con los registros únicos basados en 'uname' y 'rname'
    resultado = dataframe_sorted.drop_duplicates(subset=['uname', 'rname'], keep='first')

    return resultado

In [172]:
str_dataset = "AMZ"
type_al = "Ex"
filename = "../data/"+str_dataset+"/"+str_dataset+"-"+type_al+".csv"

acc_log = pd.read_csv(filename)

# Remove first column (is the old index)
if not str_dataset in ["AMZ", "ML"]:
    acc_log = acc_log[acc_log.columns[1:]]

# Change the uname string values to int values
if not str_dataset in ["AMZ","ML", "UN", "PM", "IoT"]:
    acc_log = acc_log.replace(acc_log.uname.unique(), range(len(acc_log.uname.unique())))

# Change the resource id
if not str_dataset in ["UN", "PM", "AMZ", "ML", "IoT"]:
    acc_log["rname"] = acc_log["rname"] + max(acc_log.uname.unique()) + 1

print("Done!")
print(acc_log.head(5)) # Show the first five rows
print("|L|=", len(acc_log))
print("Permit/Deny\n", acc_log["ACTION"].value_counts())

# Delete some variables
del filename

Done!
   Unnamed: 0  ACTION  rname  MGR_ID  ROLE_ROLLUP_1  ROLE_ROLLUP_2  \
0         261       1  39353   85475         117961         118300   
1           0       1  39353   85475         117961         118300   
2         261       1  39353   85475         117961         118300   
3           0       1  39353   85475         117961         118300   
4           0       1  39353   85475         117961         118300   

   ROLE_DEPTNAME  ROLE_TITLE  ROLE_FAMILY_DESC  ROLE_FAMILY  ROLE_CODE  uname  
0         120410      118784            117906       290919     118786    255  
1         123472      117905            117906       290919     117908      1  
2         120410      118784            117906       290919     118786    255  
3         123472      117905            117906       290919     117908      1  
4         123472      117905            117906       290919     117908      1  
|L|= 25736
Permit/Deny
 1    24253
0     1483
Name: ACTION, dtype: int64


In [179]:
# Llama a la función para cambiar los valores preservando la proporción
cambio_porcentaje = 0.10
df_modificado, cantidad_cambios = cambiar_valores_preservando_proporcion(
    acc_log.copy(), "ACTION", cambio_porcentaje)
print(df_modificado.info())
print(df_modificado["ACTION"].value_counts())

2573
No hay suficientes filas en una o ambas clases para cambiar.


TypeError: cannot unpack non-iterable NoneType object

In [177]:
### EJEMPLO REAL

##### ***** GENERAR LA COLUMNA FREQUENCIA ***** #####
df = df_modificado[["uname","rname", "ACTION"]]
frecuencias = df.groupby(["uname", "rname", "ACTION"]).size()
# Mapear las frecuencias al DataFrame original
df["freq"] = df.set_index(["uname", "rname", "ACTION"]).index.map(frecuencias).fillna(0).astype(int)
df["freq"] = df["freq"] / len(df)
df = df.drop_duplicates()
print(df, sum(df["freq"]))

##### ***** HACER LA LIMPIEZA ***** #####
resultado = quedarse_con_mayor_freq(df)
print(resultado)

       uname   rname  ACTION      freq
0        255   39353       1  0.000233
1          1   39353       1  0.000427
17      3550   17183       1  0.000039
18      1295   17183       1  0.000078
19         2   17183       1  0.000039
...      ...     ...     ...       ...
25727    507  103867       1  0.000039
25728    413   26980       1  0.000078
25730   3384   75915       1  0.000078
25732   9561   32529       1  0.000078
25734   6016   16354       1  0.000078

[10962 rows x 4 columns] 1.0000000000000249
      uname  rname  ACTION      freq
740      52  80263       1  0.000622
942      66  60006       1  0.000622
327      23  92885       1  0.000622
407      29  78098       1  0.000622
503      35  78282       1  0.000622
...     ...    ...     ...       ...
8222    729  15369       1  0.000039
8224   1343  15369       1  0.000039
8228   6634  15369       1  0.000039
8230   3995  15369       1  0.000039
6495   5897  41758       1  0.000039

[10165 rows x 4 columns]


c:\Users\danie\anaconda3\envs\tesis_env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
c:\Users\danie\anaconda3\envs\tesis_env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [178]:
##### ***** HACER LA EVALUACIÓN DE CUANTOS SON FP ***** #####

result_dataframe = compare_dataframes(resultado, acc_log)
print("Registros con ruido", cantidad_cambios)
print("Registros incorrectos", len(result_dataframe), (len(result_dataframe)/cantidad_cambios)*100)
print("Registros Correctos", cantidad_cambios - len(result_dataframe), ((cantidad_cambios - len(result_dataframe))/cantidad_cambios)*100)
print(result_dataframe)


Registros con ruido 1286
Registros incorrectos 836 65.00777604976672
Registros Correctos 450 34.99222395023328
       uname  rname  ACTION_df1      freq  Unnamed: 0  ACTION_df2  MGR_ID  \
3748    3092  18411           1  0.000272       28934           0   16589   
3749    3092  18411           1  0.000272       28934           0   16589   
3750    3092  18411           1  0.000272       28934           0   16589   
3751    3092  18411           1  0.000272       28934           0   16589   
3752    3092  18411           1  0.000272       28934           0   16589   
...      ...    ...         ...       ...         ...         ...     ...   
25662   2232  39329           1  0.000039        6921           0   27759   
25667   1436  34577           0  0.000039        7308           1   44024   
25668   1436  34577           0  0.000039        7308           1   44024   
25687   6167  37260           1  0.000039       31938           0   48340   
25714    291    998           1  0.000039 

In [11]:
acc_log_freq = df_modificado.value_counts(["uname", "rname"], normalize=True)
acc_log_freq

uname  rname
30     2533     0.010301
       2532     0.005245
       2530     0.005016
       2529     0.003926
       2531     0.003880
                  ...   
16     3494     0.000007
       3487     0.000007
       3473     0.000007
       3471     0.000007
20     1704     0.000007
Length: 34317, dtype: float64

In [7]:
acc_log_freq = df_modificado.value_counts(["uname", "rname", "ACTION"], normalize=True)
acc_log_freq

uname  rname  ACTION
30     2533   1         0.007668
       2532   1         0.003815
       2530   1         0.003625
       2531   1         0.002913
29     2533   0         0.002854
                          ...   
23     598    0         0.000007
12     1835   0         0.000007
23     594    0         0.000007
12     1838   0         0.000007
0      115    1         0.000007
Length: 40432, dtype: float64

In [35]:
### Generate biparite graph
acc_log_sub = df_modificado[df_modificado.ACTION == 1]
acc_log_freq = acc_log_sub.value_counts(["uname", "rname", "flag"], normalize=True)
test = pd.DataFrame(list(acc_log_freq.keys()), columns=["uname", "rname", "flag"])
test["weight"] = acc_log_freq.values
azucar = eliminar_repetidos_con_mayor_peso(test)
azucar

,uname,rname,flag,weight
0,30,2533,0,0.016200
1,30,2532,0,0.009720
2,30,2530,0,0.008250
3,30,2529,0,0.007449
4,30,2531,0,0.006614
...,...,...,...,...
7835,0,2050,0,0.000033
7839,1,4411,0,0.000033
7840,1,4611,0,0.000033
7841,1,4559,0,0.000033


In [19]:

def compare_dataframes(df1, df2):
    """
    Compare two DataFrames based on specified columns.

    Args:
        df1 (pd.DataFrame): First DataFrame.
        df2 (pd.DataFrame): Second DataFrame.

    Returns:
        pd.DataFrame: Resulting DataFrame with rows where df1.ACTION != df2.ACTION.
    """
    # Merge DataFrames on "uname" and "rname"
    merged_df = pd.merge(df1, df2, on=["uname", "rname"], suffixes=('_df1', '_df2'))

    # Filter rows where ACTION values are different
    result_df = merged_df[merged_df['ACTION_df1'] != merged_df['ACTION_df2']]

    return result_df

# Example usage:
df1 = pd.DataFrame({
    "uname": [1, 2, 3],
    "rname": [10, 20, 30],
    "ACTION": [0, 1, 0]
})

df2 = pd.DataFrame({
    "uname": [1, 2, 3],
    "rname": [10, 20, 30],
    "ACTION": [1, 1, 0]
})

result_dataframe = compare_dataframes(df1, df2)
print(result_dataframe)


   uname  rname  ACTION_df1  ACTION_df2
0      1     10           0           1
